In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from pyomo.network import Arc, SequentialDecomposition

from idaes.core.util.initialization import propagate_state

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np
import pandas as pd

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

# Import R32, R125, IL info file
from HFCs_bmimPF6_PR import configuration

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.state_block = m1.fs.properties.state_block_class(
            default={"parameters": m1.fs.properties,
                     "defined_state": True})

m2 = ConcreteModel()

m2.fs = FlowsheetBlock(default={"dynamic": False})

m2.fs.properties = GenericParameterBlock(default=configuration)

m2.fs.state_block = m2.fs.properties.state_block_class(
            default={"parameters": m2.fs.properties,
                     "defined_state": True})

m3 = ConcreteModel()

m3.fs = FlowsheetBlock(default={"dynamic": False})

m3.fs.properties = GenericParameterBlock(default=configuration)

m3.fs.state_block = m3.fs.properties.state_block_class(
            default={"parameters": m3.fs.properties,
                     "defined_state": True})

m4 = ConcreteModel()

m4.fs = FlowsheetBlock(default={"dynamic": False})

m4.fs.properties = GenericParameterBlock(default=configuration)

m4.fs.state_block = m4.fs.properties.state_block_class(
            default={"parameters": m4.fs.properties,
                     "defined_state": True})

In [3]:
# Constant M1
m1.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
m1.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
m1.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.02608)
m1.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.0704)
m1.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(0.345)
m1.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(0.058856)

# Linear M2
m2.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.39255)
m2.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.375437)
m2.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(0.02700)
m2.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(0.26803)
m2.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(-1.601578)
m2.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.111417)
m2.fs.properties.PR_kappa_B['R32', 'R125'].fix(-0.389117)
m2.fs.properties.PR_kappa_B['R125', 'R32'].fix(0.3892)
m2.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(-0.04879)
m2.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(-0.3153)
m2.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(2.086037)
m2.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0.170299)

# Ternary M3
m3.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
m3.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
m3.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.043507)
m3.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.35057)
m3.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(1.677866)
m3.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(0.060435)

# Ternary M4
# m4.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
# m4.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
# m4.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.0328315)
# m4.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.1130500)
# m4.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(0.413767)
# m4.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(0.059460)

# Linear M5
# m4.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.39255)
# m4.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.375437)
# m4.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.04108)
# m4.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.1353)
# m4.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(-1.4214)
# m4.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.1557)
# m4.fs.properties.PR_kappa_B['R32', 'R125'].fix(-0.389117)
# m4.fs.properties.PR_kappa_B['R125', 'R32'].fix(0.3892)
# m4.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(0.0098)
# m4.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(0.0303)
# m4.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(1.7749)
# m4.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0.2070)

# m4.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.39255)
# m4.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.375437)
# m4.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.0235)
# m4.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(1.58052)
# m4.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(-0.9793)
# m4.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.129826)
# m4.fs.properties.PR_kappa_B['R32', 'R125'].fix(-0.389117)
# m4.fs.properties.PR_kappa_B['R125', 'R32'].fix(0.3892)
# m4.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(-0.01365)
# m4.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(-1.7695)
# m4.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(1.42148)
# m4.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0.18474)

m4.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.39255)
m4.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.375437)
m4.fs.properties.PR_kappa_B['R32', 'R125'].fix(-0.389117)
m4.fs.properties.PR_kappa_B['R125', 'R32'].fix(0.3892)

m4.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(-1.764584)
m4.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.161157)
m4.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.010645)
m4.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.024842)

m4.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(2.123490)
m4.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0.21234)
m4.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(-0.09102)
m4.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(-0.00556)


In [4]:
solver = SolverFactory('ipopt')
# solver.options = {'tol': 1e-6}

In [5]:
# Set pressures to loop over
# R125_z = [0.50]
R125_z = [0.001,0.25,0.5,0.75,0.99]


# Make outlet arrays
IL_x = np.zeros(len(R125_z))
IL_y = np.zeros(len(R125_z))
R32_x = np.zeros(len(R125_z))
R32_y = np.zeros(len(R125_z))
R125_x = np.zeros(len(R125_z))
R125_y = np.zeros(len(R125_z))

IL_x2 = np.zeros(len(R125_z))
IL_y2 = np.zeros(len(R125_z))
R32_x2 = np.zeros(len(R125_z))
R32_y2 = np.zeros(len(R125_z))
R125_x2 = np.zeros(len(R125_z))
R125_y2 = np.zeros(len(R125_z))

IL_x3 = np.zeros(len(R125_z))
IL_y3 = np.zeros(len(R125_z))
R32_x3 = np.zeros(len(R125_z))
R32_y3 = np.zeros(len(R125_z))
R125_x3 = np.zeros(len(R125_z))
R125_y3 = np.zeros(len(R125_z))
L3 = np.zeros(len(R125_z))
V3 = np.zeros(len(R125_z))

IL_x4 = np.zeros(len(R125_z))
IL_y4 = np.zeros(len(R125_z))
R32_x4 = np.zeros(len(R125_z))
R32_y4 = np.zeros(len(R125_z))
R125_x4 = np.zeros(len(R125_z))
R125_y4 = np.zeros(len(R125_z))
L4 = np.zeros(len(R125_z))
V4 = np.zeros(len(R125_z))

m1.fs.state_block.flow_mol.fix(10)
m1.fs.state_block.temperature.fix(298)
m1.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.9) 
m1.fs.state_block.pressure.fix(100000)

m2.fs.state_block.flow_mol.fix(10)
m2.fs.state_block.temperature.fix(298)
m2.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.9) 
m2.fs.state_block.pressure.fix(100000)

m3.fs.state_block.flow_mol.fix(10)
m3.fs.state_block.temperature.fix(298)
m3.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.9) 
m3.fs.state_block.pressure.fix(100000)

m4.fs.state_block.flow_mol.fix(10)
m4.fs.state_block.temperature.fix(298)
m4.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.9) 
m4.fs.state_block.pressure.fix(100000)

m1.fs.liq = Param(mutable=True, default=0.3)
m1.fs.liquid =  Constraint(expr=m1.fs.state_block.mole_frac_phase_comp['Vap', "R32"] == m1.fs.liq)
m1.fs.liquid2 =  Constraint(expr=m1.fs.state_block.mole_frac_phase_comp['Vap', "R125"] == 1-m1.fs.liq)

m2.fs.liq = Param(mutable=True, default=0.3)
m2.fs.liquid =  Constraint(expr=m2.fs.state_block.mole_frac_phase_comp['Vap', "R32"] == m2.fs.liq)
m2.fs.liquid2 =  Constraint(expr=m2.fs.state_block.mole_frac_phase_comp['Vap', "R125"] == 1-m2.fs.liq)

m3.fs.liq = Param(mutable=True, default=0.3)
m3.fs.liquid =  Constraint(expr=m3.fs.state_block.mole_frac_phase_comp['Vap', "R32"] == m3.fs.liq)
m3.fs.liquid2 =  Constraint(expr=m3.fs.state_block.mole_frac_phase_comp['Vap', "R125"] == 1-m3.fs.liq)

m4.fs.liq = Param(mutable=True, default=0.3)
m4.fs.liquid =  Constraint(expr=m4.fs.state_block.mole_frac_phase_comp['Vap', "R32"] == m4.fs.liq)
m4.fs.liquid2 =  Constraint(expr=m4.fs.state_block.mole_frac_phase_comp['Vap', "R125"] == 1-m4.fs.liq)

for i in range(len(R125_z)):
    
    m1.fs.state_block.flow_mol.fix(10)
    m1.fs.state_block.temperature.fix(298)
    m1.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.7) 
    m1.fs.state_block.pressure.fix(100000)

    m2.fs.state_block.flow_mol.fix(10)
    m2.fs.state_block.temperature.fix(298)
    m2.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.3) 
    m2.fs.state_block.pressure.fix(100000)

    m3.fs.state_block.flow_mol.fix(10)
    m3.fs.state_block.temperature.fix(298)
    m3.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.7) 
    m3.fs.state_block.pressure.fix(100000)

    m4.fs.state_block.flow_mol.fix(10)
    m4.fs.state_block.temperature.fix(298)
    m4.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.7) 
    m4.fs.state_block.pressure.fix(100000)
    
    m1.fs.liq = R125_z[i]
    m2.fs.liq = R125_z[i]
    m3.fs.liq = R125_z[i]
    m4.fs.liq = R125_z[i]

    m1.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)              
    m2.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)
    m3.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)
    m4.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)
    
    # solve the model
    status1 = solver.solve(m1, tee = False)
    status2 = solver.solve(m2, tee = False)
    status3 = solver.solve(m3, tee = False)
    status4 = solver.solve(m4, tee = False)
        
    # If solution is optimal store the concentration, and calculated temperatures in the created arrays
    if (status1.solver.status == SolverStatus.ok) and (status1.solver.termination_condition == TerminationCondition.optimal):

        IL_x[i] = value(m1.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"])
        IL_y[i] = 1e-8
        R32_x[i] = value(m1.fs.state_block.mole_frac_phase_comp['Liq', "R32"]) 
        R32_y[i] = value(m1.fs.state_block.mole_frac_phase_comp['Vap', "R32"])
        R125_x[i] = value(m1.fs.state_block.mole_frac_phase_comp['Liq', "R125"])
        R125_y[i] = value(m1.fs.state_block.mole_frac_phase_comp['Vap', "R125"])

    if (status2.solver.status == SolverStatus.ok) and (status2.solver.termination_condition == TerminationCondition.optimal):
        IL_x2[i] = value(m2.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"])
        IL_y2[i] = 1e-8
        R32_x2[i] = value(m2.fs.state_block.mole_frac_phase_comp['Liq', "R32"]) 
        R32_y2[i] = value(m2.fs.state_block.mole_frac_phase_comp['Vap', "R32"])
        R125_x2[i] = value(m2.fs.state_block.mole_frac_phase_comp['Liq', "R125"])
        R125_y2[i] = value(m2.fs.state_block.mole_frac_phase_comp['Vap', "R125"])

    if (status3.solver.status == SolverStatus.ok) and (status3.solver.termination_condition == TerminationCondition.optimal):  

        IL_x3[i] = value(m3.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"])
        IL_y3[i] = 1e-8
        R32_x3[i] = value(m3.fs.state_block.mole_frac_phase_comp['Liq', "R32"]) 
        R32_y3[i] = value(m3.fs.state_block.mole_frac_phase_comp['Vap', "R32"])
        R125_x3[i] = value(m3.fs.state_block.mole_frac_phase_comp['Liq', "R125"])
        R125_y3[i] = value(m3.fs.state_block.mole_frac_phase_comp['Vap', "R125"])

    if (status4.solver.status == SolverStatus.ok) and (status4.solver.termination_condition == TerminationCondition.optimal):  

        IL_x4[i] = value(m4.fs.state_block.mole_frac_phase_comp['Liq', "bmimPF6"])
        IL_y4[i] = 1e-8
        R32_x4[i] = value(m4.fs.state_block.mole_frac_phase_comp['Liq', "R32"]) 
        R32_y4[i] = value(m4.fs.state_block.mole_frac_phase_comp['Vap', "R32"])
        R125_x4[i] = value(m4.fs.state_block.mole_frac_phase_comp['Liq', "R125"])
        R125_y4[i] = value(m4.fs.state_block.mole_frac_phase_comp['Vap', "R125"])        
        #Change this to feasible
        print("Running R125:",R125_x[i])

    else:
        print('Infeasible.')

2022-03-16 16:25:17 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2022-03-16 16:25:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:25:21 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2022-03-16 16:25:23 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
Running R125: 0.026514713893277177
2022-03-16 16:25:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:25:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-16 16:25:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal 

In [6]:
print(R32_x)
print(R32_y)
print(R125_x)
print(R125_y)
print(IL_x)
print(IL_y)
print(R32_x2)
print(R32_y2)
print(R125_x2)
print(R125_y2)
print(IL_x2)
print(IL_y2)
print(R32_x3)
print(R32_y3)
print(R125_x3)
print(R125_y3)
print(IL_x3)
print(IL_y3)
print(R32_x4)
print(R32_y4)
print(R125_x4)
print(R125_y4)
print(IL_x4)
print(IL_y4)

[8.02026022e-05 1.99566388e-02 3.97275836e-02 5.93173126e-02
 7.79569378e-02]
[0.001 0.25  0.5   0.75  0.99 ]
[0.02651471 0.01980948 0.01314276 0.00654009 0.00026042]
[0.999 0.75  0.5   0.25  0.01 ]
[0.97340508 0.96023388 0.94712966 0.9341426  0.92178264]
[1.e-08 1.e-08 1.e-08 1.e-08 1.e-08]
[7.75845239e-05 1.92977505e-02 3.84015340e-02 5.73165120e-02
 7.53018841e-02]
[0.001 0.25  0.5   0.75  0.99 ]
[0.02667537 0.01991027 0.01319697 0.00656083 0.00026101]
[0.999 0.75  0.5   0.25  0.01 ]
[0.97324704 0.96079198 0.9484015  0.93612265 0.92443711]
[1.e-08 1.e-08 1.e-08 1.e-08 1.e-08]
[9.65730472e-05 2.36450211e-02 4.63393060e-02 6.81517040e-02
 8.83193911e-02]
[0.001 0.25  0.5   0.75  0.99 ]
[0.02751451 0.02010213 0.01305152 0.00636051 0.00024841]
[0.999 0.75  0.5   0.25  0.01 ]
[0.97238891 0.95625285 0.94060917 0.92548779 0.9114322 ]
[1.e-08 1.e-08 1.e-08 1.e-08 1.e-08]
[8.33242114e-05 2.07115133e-02 4.11868465e-02 6.14316183e-02
 8.06546911e-02]
[0.001 0.25  0.5   0.75  0.99 ]
[0.02880797

In [7]:
df = pd.DataFrame({'IL_x':IL_x, 'R32_x':R32_x,'R125_x':R125_x,'IL_y':IL_y,'R32_y':R32_y,'R125_y':R125_y,
                   'IL_x2':IL_x2,'R32_x2':R32_x2,'R125_x2':R125_x2,'IL_y2':IL_y2,'R32_y2':R32_y2,'R125_y2':R125_y2,
                    'IL_x3':IL_x3,'R32_x3':R32_x3,'R125_x3':R125_x3,'IL_y3':IL_y3,'R32_y3':R32_y3,'R125_y3':R125_y3,
                    'IL_x4':IL_x4,'R32_x4':R32_x4,'R125_x4':R125_x4,'IL_y4':IL_y4,'R32_y4':R32_y4,'R125_y4':R125_y4})


In [8]:
df.to_csv('ku_250_bw3.csv')